In [36]:
import pandas as pd
test_data=pd.read_csv("/notebooks/test_prediction.csv")
test_data.columns=["id","clip_count"]
test_features=pd.read_csv("/notebooks/test_features.csv")[-5000:]
test_features.columns=["id","True count","size","grey_pixel_number","50"]
display(test_data,test_features)

,id,clip_count
0,45001,68.247750
1,45002,40.110317
2,45003,41.237000
3,45004,38.798252
4,45005,18.474697
...,...,...
4995,49996,35.309082
4996,49997,53.150887
4997,49998,71.147865
4998,49999,23.043373


,id,True count,size,grey_pixel_number,50
15000,45001,68.0,61355.0,14280.0,51635.0
15001,45002,40.0,40761.0,9209.0,56548.0
15002,45003,41.0,45467.0,9829.0,55936.0
15003,45004,39.0,37876.0,8451.0,57317.0
15004,45005,19.0,18560.0,3932.0,61734.0
...,...,...,...,...,...
19995,49996,35.0,35775.0,7886.0,57833.0
19996,49997,54.0,50962.0,11382.0,54432.0
19997,49998,72.0,63938.0,15244.0,50704.0
19998,49999,24.0,24989.0,5269.0,60416.0


In [68]:
data=pd.concat([test_features.reset_index(drop=True),test_data["clip_count"].reset_index(drop=True)],axis=1)
data["Error"]=data["True count"]-data["clip_count"]
data

,id,True count,size,grey_pixel_number,50,clip_count,Error
0,45001,68.0,61355.0,14280.0,51635.0,68.247750,-0.247750
1,45002,40.0,40761.0,9209.0,56548.0,40.110317,-0.110317
2,45003,41.0,45467.0,9829.0,55936.0,41.237000,-0.237000
3,45004,39.0,37876.0,8451.0,57317.0,38.798252,0.201748
4,45005,19.0,18560.0,3932.0,61734.0,18.474697,0.525303
...,...,...,...,...,...,...,...
4995,49996,35.0,35775.0,7886.0,57833.0,35.309082,-0.309082
4996,49997,54.0,50962.0,11382.0,54432.0,53.150887,0.849113
4997,49998,72.0,63938.0,15244.0,50704.0,71.147865,0.852135
4998,49999,24.0,24989.0,5269.0,60416.0,23.043373,0.956627


In [87]:
print("Error to be corrected: " + str(data["Error"].abs().mean()))


Error to be corrected: 0.5469445792940001


In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(data.iloc[:,2:-2],
                                                  data.iloc[:,-1],
                                                  test_size=0.2)

import xgboost as xgb
dtrain=xgb.DMatrix(X_train, label=y_train)
dtest=xgb.DMatrix(X_test, label=y_test)

param = {'max_depth':3, 'eta':0.3, 'silent':1, 
         'objective':'reg:squarederror','eval_metric':"mae"}
evals = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(param, dtrain, num_boost_round=70, evals=evals)

[0]	eval-mae:0.61836	train-mae:0.62648
[1]	eval-mae:0.58483	train-mae:0.58986
[2]	eval-mae:0.56751	train-mae:0.56974
[3]	eval-mae:0.55862	train-mae:0.55721
[4]	eval-mae:0.55389	train-mae:0.55074
[5]	eval-mae:0.55113	train-mae:0.54509
[6]	eval-mae:0.54955	train-mae:0.54274
[7]	eval-mae:0.54765	train-mae:0.54028
[8]	eval-mae:0.54783	train-mae:0.53783
[9]	eval-mae:0.54719	train-mae:0.53608
[10]	eval-mae:0.54763	train-mae:0.53438
[11]	eval-mae:0.54734	train-mae:0.53373
[12]	eval-mae:0.54744	train-mae:0.53258
[13]	eval-mae:0.54800	train-mae:0.53154
[14]	eval-mae:0.54786	train-mae:0.53130
[15]	eval-mae:0.54777	train-mae:0.53052
[16]	eval-mae:0.54857	train-mae:0.52950
[17]	eval-mae:0.54885	train-mae:0.52852
[18]	eval-mae:0.54898	train-mae:0.52761
[19]	eval-mae:0.54928	train-mae:0.52637
[20]	eval-mae:0.55050	train-mae:0.52424
[21]	eval-mae:0.55050	train-mae:0.52394
[22]	eval-mae:0.55079	train-mae:0.52253
[23]	eval-mae:0.55107	train-mae:0.52167
[24]	eval-mae:0.55108	train-mae:0.52118
[25]	eval-